PROBLEMS:
* (emdat) we don't have the day or even the month of some flood event (we will use the in the merge)
* (precipitation) we don't have the precipitation data until 2020 for all the countries (we will use the flood events for these countires during the merge)
* (precipitation) we don't have the precipitation for every day 
* (emdat) we don't always have the number of affected 
=> this causes a lot of loss in the data during merge 

SOLUTIONS:
*  solution to missing days/ months for emdat events : I filled these missing values by random values (of day/month) 
*  solution to the missing number of affected: I filled the missing values by the mean values per region per year (or total mean value if the last value is misssing too )
* 
* 

TO DO 
* find a solution to the missing precipitation data 
* normalization 
* resampling 
* feature selection algorithm (there is a feature importance score included in RF)
* using historical flood data as new features (like the number of floods in the country during the last years ... ) 

In [110]:
import pandas as pd 
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import math
import random 

# Precipitation

In [2]:
#read precipitation data 
precipitation = pd.read_csv('daily_rain_by_country_feature.csv.gz', compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)

In [3]:
pd.set_option('display.max_columns', None)

precipitation.head()

,country_ISO3,date,avg_rain,sum_rain,max_rain,stddev_rain,station_count,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_sum,last_10_days_avg,last_10_days_max,last_20_days_sum,last_20_days_avg,last_20_days_max
0,CCK,1949-08-19,30.0,30.0,30.0,0.0,1,60.0,30.0,30.0,60.0,30.0,30.0,60.0,30.0,30.0
1,CCK,1949-08-19,30.0,30.0,30.0,0.0,1,60.0,30.0,30.0,60.0,30.0,30.0,60.0,30.0,30.0
2,CCK,1949-09-12,89.0,89.0,89.0,0.0,1,178.0,89.0,89.0,178.0,89.0,89.0,178.0,89.0,89.0
3,CCK,1949-09-12,89.0,89.0,89.0,0.0,1,178.0,89.0,89.0,178.0,89.0,89.0,178.0,89.0,89.0
4,CCK,1950-04-12,51.0,51.0,51.0,0.0,1,102.0,51.0,51.0,102.0,51.0,51.0,102.0,51.0,51.0


In [4]:
precipitation.tail()

,country_ISO3,date,avg_rain,sum_rain,max_rain,stddev_rain,station_count,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_sum,last_10_days_avg,last_10_days_max,last_20_days_sum,last_20_days_avg,last_20_days_max
10225713,ATF,2018-12-20,0.0,0.0,0.0,0.0,1,122.0,10.166667,56.0,448.0,20.363636,160.0,672.0,16.000000,160.0
10225714,ATF,2018-12-21,0.0,0.0,0.0,0.0,1,112.0,9.333333,56.0,448.0,20.363636,160.0,672.0,16.000000,160.0
10225715,ATF,2018-12-21,0.0,0.0,0.0,0.0,1,112.0,9.333333,56.0,448.0,20.363636,160.0,672.0,16.000000,160.0
10225716,ATF,2018-12-22,15.0,15.0,15.0,0.0,1,30.0,2.500000,15.0,478.0,21.727273,160.0,686.0,16.333333,160.0
10225717,ATF,2018-12-22,15.0,15.0,15.0,0.0,1,30.0,2.500000,15.0,478.0,21.727273,160.0,686.0,16.333333,160.0


In [5]:
print(precipitation.shape)
print(precipitation.columns)

(10225718, 16)
Index(['country_ISO3', 'date', ' avg_rain', 'sum_rain', 'max_rain',
       'stddev_rain', 'station_count', 'last_5_days_sum', 'last_5_days_avg',
       'last_5_days_max', 'last_10_days_sum', 'last_10_days_avg',
       'last_10_days_max', 'last_20_days_sum', 'last_20_days_avg',
       'last_20_days_max'],
      dtype='object')


In [6]:
# check is we have null dates 
precipitation[precipitation['date'].isnull()]

,country_ISO3,date,avg_rain,sum_rain,max_rain,stddev_rain,station_count,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_sum,last_10_days_avg,last_10_days_max,last_20_days_sum,last_20_days_avg,last_20_days_max


In [7]:
# convert the date to datetime type 
precipitation['date']=pd.to_datetime(precipitation['date'])
precipitation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10225718 entries, 0 to 10225717
Data columns (total 16 columns):
country_ISO3        object
date                datetime64[ns]
 avg_rain           float64
sum_rain            float64
max_rain            float64
stddev_rain         float64
station_count       int64
last_5_days_sum     float64
last_5_days_avg     float64
last_5_days_max     float64
last_10_days_sum    float64
last_10_days_avg    float64
last_10_days_max    float64
last_20_days_sum    float64
last_20_days_avg    float64
last_20_days_max    float64
dtypes: datetime64[ns](1), float64(13), int64(1), object(1)
memory usage: 1.2+ GB


In [8]:
# split the date into day , month and year 
precipitation['Start Day'] = precipitation.date.dt.day
precipitation['Start Month'] = precipitation.date.dt.month
precipitation['Start Year'] = precipitation.date.dt.year

In [9]:
precipitation.head()

,country_ISO3,date,avg_rain,sum_rain,max_rain,stddev_rain,station_count,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_sum,last_10_days_avg,last_10_days_max,last_20_days_sum,last_20_days_avg,last_20_days_max,Start Day,Start Month,Start Year
0,CCK,1949-08-19,30.0,30.0,30.0,0.0,1,60.0,30.0,30.0,60.0,30.0,30.0,60.0,30.0,30.0,19,8,1949
1,CCK,1949-08-19,30.0,30.0,30.0,0.0,1,60.0,30.0,30.0,60.0,30.0,30.0,60.0,30.0,30.0,19,8,1949
2,CCK,1949-09-12,89.0,89.0,89.0,0.0,1,178.0,89.0,89.0,178.0,89.0,89.0,178.0,89.0,89.0,12,9,1949
3,CCK,1949-09-12,89.0,89.0,89.0,0.0,1,178.0,89.0,89.0,178.0,89.0,89.0,178.0,89.0,89.0,12,9,1949
4,CCK,1950-04-12,51.0,51.0,51.0,0.0,1,102.0,51.0,51.0,102.0,51.0,51.0,102.0,51.0,51.0,12,4,1950


In [10]:
# rename columns 
precipitation.columns=np.array(['ISO', 'date', ' avg_rain', 'sum_rain', 'max_rain',
       'stddev_rain', 'station_count', 'last_5_days_sum', 'last_5_days_avg',
       'last_5_days_max', 'last_10_days_sum', 'last_10_days_avg',
       'last_10_days_max', 'last_20_days_sum', 'last_20_days_avg',
       'last_20_days_max', 'Start Day', 'Start Month', 'Start Year'])

In [11]:
## remove duplicate date in NOAA that lead to more columns after merge with EMDAT
precipitation=precipitation[~precipitation.duplicated(keep='first')]
precipitation.shape

(5112859, 19)

# EMDAT 

In [39]:
table= pd.read_excel('emdat_public_2020_09_12_query_uid-tAnKEX.xlsx',skiprows=6)
# Select floods 
flood_table=table[table['Disaster Type']=='Flood']
#delete columns with nans only 
for column in flood_table.columns:
    if flood_table[column].isnull().all():
        del flood_table[column]

In [40]:
flood_table.head(40)

,Dis No,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,Entry Criteria,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,OFDA Response,Appeal,Declaration,Aid Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI
16,1906-0023-BEL,1906,23,Natural,Hydrological,Flood,NaN,NaN,Govern,Belgium,BEL,Western Europe,Europe,Louvain region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1906,5.0,14.0,1906,5.0,14.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.522300
17,1906-0024-BEL,1906,24,Natural,Hydrological,Flood,NaN,NaN,Govern,Belgium,BEL,Western Europe,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1906,4.0,NaN,1906,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.522300
50,1911-0002-CHN,1911,2,Natural,Hydrological,Flood,Riverine flood,NaN,Kill,China,CHN,Eastern Asia,Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1911,NaN,NaN,1911,NaN,NaN,100000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.652756
63,1915-0011-CHN,1915,11,Natural,Hydrological,Flood,Riverine flood,NaN,Affect,China,CHN,Eastern Asia,Asia,Guangzhou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,Km2,NaN,NaN,NaN,1915,NaN,NaN,1915,NaN,NaN,NaN,NaN,3000000.0,NaN,3000000.0,NaN,NaN,NaN,3.965849
79,1917-0013-CHN,1917,13,Natural,Hydrological,Flood,Riverine flood,NaN,Affect,China,CHN,Eastern Asia,Asia,Tianjin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1917,NaN,NaN,1917,NaN,NaN,NaN,NaN,635000.0,NaN,635000.0,NaN,NaN,NaN,5.022539
101,1920-0005-DFR,1920,5,Natural,Hydrological,Flood,NaN,NaN,Declar,Germany Fed Rep,DFR,Western Europe,Europe,Danube area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1920,8.0,NaN,1920,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20000.0,7.853425
128,1925-0007-DFR,1925,7,Natural,Hydrological,Flood,NaN,NaN,Declar,Germany Fed Rep,DFR,Western Europe,Europe,Rhine area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1925,12.0,NaN,1925,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19000.0,6.875008
130,1926-0018-BEL,1926,18,Natural,Hydrological,Flood,Riverine flood,NaN,Govern,Belgium,BEL,Western Europe,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1926,NaN,NaN,1926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.940236
135,1926-0006-DFR,1926,6,Natural,Hydrological,Flood,NaN,NaN,Declar,Germany Fed Rep,DFR,Western Europe,Europe,Rhine area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1926,6.0,NaN,1926,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31000.0,6.940236
138,1926-0008-IND,1926,8,Natural,Hydrological,Flood,Riverine flood,NaN,OFDA,India,IND,Southern Asia,Asia,Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1926,7.0,19.0,1926,7.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.940236


In [41]:
print(flood_table.shape)
flood_table.columns

(5277, 41)


Index(['Dis No', 'Year', 'Seq', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Event Name', 'Entry Criteria',
       'Country', 'ISO', 'Region', 'Continent', 'Location', 'Origin',
       'Associated Dis', 'Associated Dis2', 'OFDA Response', 'Appeal',
       'Declaration', 'Aid Contribution', 'Dis Mag Value', 'Dis Mag Scale',
       'Latitude', 'Longitude', 'River Basin', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths',
       'No Injured', 'No Affected', 'No Homeless', 'Total Affected',
       'Reconstruction Costs ('000 US$)', 'Insured Damages ('000 US$)',
       'Total Damages ('000 US$)', 'CPI'],
      dtype='object')

In [42]:
print('events with unknown start year')
print(flood_table[flood_table['Start Year'].isnull()].shape[0])
print('events with unknown start month')
print(flood_table[flood_table['Start Month'].isnull()].shape[0])
print('events with unknown start day')
print(flood_table[flood_table['Start Day'].isnull()].shape[0])
print('events with unknown end Year')
print(flood_table[flood_table['End Year'].isnull()].shape[0])
print('events with unknown end month')
print(flood_table[flood_table['End Month'].isnull()].shape[0])
print('events with unknown end day')
print(flood_table[flood_table['End Day'].isnull()].shape[0])
### 929 events don't have the exact date and can't have the precipitation information
print('events with unknown start month but known end month')
print(flood_table[flood_table['Start Month'].isnull() & flood_table['End Month'].notnull()].shape[0])
print('events with unknown start day but known end day')
print(flood_table[flood_table['Start Day'].isnull() & flood_table['End Day'].notnull()].shape[0])


events with unknown start year
0
events with unknown start month
51
events with unknown start day
929
events with unknown end Year
0
events with unknown end month
50
events with unknown end day
941
events with unknown start month but known end month
1
events with unknown start day but known end day
17


### handling missing dates
 if we have the end day (end month) and we don't have the start day (start month), we use the values of end day (end month) instead. 


In [43]:
## if we have the end day (end month) and we don't have the start day (start month), we use the values of end day (end month) instead. 
flood_table['Start Day']=flood_table['Start Day'].fillna(flood_table['End Day'])
flood_table['Start Month']=flood_table['Start Month'].fillna(flood_table['End Month'])
print(flood_table[flood_table['Start Day'].isnull()].shape[0])
print(flood_table[flood_table['Start Month'].isnull()].shape[0])

912
50


C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


to handle missing start days/ start months where we don't know the end days/ end months:

* we can choose 15 for the day in order to assure that the error in each date is no more than ±16
* we can choose 6 for the month in order to assure that the error in each date is no more than ±7

However, according to https://journals.sagepub.com/doi/pdf/10.1177/1536867X1201200110 this is called mean imputation and it is undesirable because it produces spikes at the mean and distorts distributional parameters
such as the variance

#### So, another solution is to select random values for these missing values 

In [44]:
flood_table['Start Day']=flood_table['Start Day'].fillna(random.randint(1, 28))
## replacing missing months by the 6th of the month 
flood_table['Start Month']=flood_table['Start Month'].fillna(random.randint(1, 12))
print(flood_table[flood_table['Start Day'].isnull()].shape[0])
print(flood_table[flood_table['Start Month'].isnull()].shape[0])

0
0


C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


# Predicting the number of total affected 

In [251]:
flood_table_full_date=flood_table[flood_table['Start Day'].notnull()]
print(flood_table_full_date.shape)

(5277, 41)


In [252]:
# we can use left join if we can replace the nans in rain data (precipitation) by other values
total_data=flood_table_full_date.merge(precipitation,how='inner',left_on=['ISO','Start Year','Start Month','Start Day'],right_on=['ISO','Start Year','Start Month','Start Day'])
print(total_data.shape)
print(total_data.columns)

(4019, 56)
Index(['Dis No', 'Year', 'Seq', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Event Name', 'Entry Criteria',
       'Country', 'ISO', 'Region', 'Continent', 'Location', 'Origin',
       'Associated Dis', 'Associated Dis2', 'OFDA Response', 'Appeal',
       'Declaration', 'Aid Contribution', 'Dis Mag Value', 'Dis Mag Scale',
       'Latitude', 'Longitude', 'River Basin', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths',
       'No Injured', 'No Affected', 'No Homeless', 'Total Affected',
       'Reconstruction Costs ('000 US$)', 'Insured Damages ('000 US$)',
       'Total Damages ('000 US$)', 'CPI', 'date', ' avg_rain', 'sum_rain',
       'max_rain', 'stddev_rain', 'station_count', 'last_5_days_sum',
       'last_5_days_avg', 'last_5_days_max', 'last_10_days_sum',
       'last_10_days_avg', 'last_10_days_max', 'last_20_days_sum',
       'last_20_days_avg', 'last_20_days_max'],
      dtype

### selecting features

In [253]:
## handling missing data in total affected column :

affected_by_country_per_year=total_data[['Region','ISO','Total Affected','Start Year']].groupby(by=['Region','ISO','Start Year']).mean()
affected_by_region_per_year=total_data[['Region','Total Affected','Start Year']].groupby(by=['Region','Start Year']).mean()
affected_by_region_total=total_data[['Region','Total Affected']].groupby(by=['Region']).mean()
# if we don't have the mean number of total affected in a country for a given year
#we take the mean of the region for the given year, and if we don't have both we take the mean over all the period 
for i in affected_by_region_per_year.index:
    if math.isnan(affected_by_region_per_year.loc[i,'Total Affected']):
        affected_by_region_per_year.loc[i,'Total Affected']=affected_by_region_total.loc[i[0],'Total Affected']
        
for i in affected_by_country_per_year.index:
    if math.isnan(affected_by_country_per_year.loc[i,'Total Affected']):
        affected_by_country_per_year.loc[i,'Total Affected']=affected_by_region_per_year.loc[(i[0],i[2]),'Total Affected']
affected_by_country_per_year

Total Affected
Region                    ISO Start Year                
Australia and New Zealand AUS 1955           4495.075000
                              1971           4495.075000
                              1973          12000.000000
                              1974           4495.075000
                              1975           4495.075000
...                                                  ...
Western Europe            LUX 1993          37866.666667
                          NLD 1953         300000.000000
                              1992          15299.367647
                              1993          13000.000000
                              1998           2000.000000

[2164 rows x 1 columns]

In [ ]:
## handling missing data in total affected column :
for i in total_data.index:
    if math.isnan(total_data.loc[i,'Total Affected']):
        total_data.loc[i,'Total Affected']=affected_by_country_per_year.loc[(total_data.loc[i,'Region'],total_data.loc[i,'ISO'],total_data.loc[i,'Start Year']),'Total Affected']

In [259]:
X=total_data[['ISO','Region', 'Continent','Start Year','Entry Criteria',
       'Start Month','Start Day',' avg_rain', 'sum_rain',
       'max_rain', 'stddev_rain', 'last_5_days_sum',
       'last_5_days_avg', 'last_5_days_max', 
       'last_10_days_avg', 'last_10_days_max', 
       'last_20_days_avg', 'last_20_days_max']]

y=total_data['Total Affected']

In [260]:
y

0       1.529937e+04
1       1.529937e+04
2       1.529937e+04
3       2.604127e+06
4       7.391684e+04
            ...     
4014    6.000000e+04
4015    3.900000e+02
4016    1.800000e+04
4017    4.335000e+04
4018    9.258000e+04
Name: Total Affected, Length: 4019, dtype: float64

In [261]:
#cleaning Entry Criteria
print(X['Entry Criteria'].unique())
X['Entry Criteria']=X['Entry Criteria'].fillna('Unknown')
X['Entry Criteria']=X['Entry Criteria'].apply(lambda x: 'Unknown' if x=='--' else x)
X['Entry Criteria'].unique()

['Govern' 'OFDA' 'Kill' 'Affect' 'Declar' 'SigDam' 'Affected' 'Unknown'
 'Regional' 'Declar/Int' 'Waiting' 'SigDis' nan '--']


C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


array(['Govern', 'OFDA', 'Kill', 'Affect', 'Declar', 'SigDam', 'Affected',
       'Unknown', 'Regional', 'Declar/Int', 'Waiting', 'SigDis'],
      dtype=object)

In [262]:
# replacing ISO by numeric values 
ISO_dic={}
count=0
for iso in X['ISO'].unique():
    ISO_dic[iso]=count
    count+=1
X['ISO']=X['ISO'].apply(lambda x: ISO_dic[x])
# replacing Region by numeric values 
Region_dic={}
count=0
for region in X['Region'].unique():
    Region_dic[region]=count
    count+=1
X['Region']=X['Region'].apply(lambda x: Region_dic[x])
# replacing Continent by numeric values 
Continent_dic={}
count=0
for continent in X['Continent'].unique():
    Continent_dic[continent]=count
    count+=1
X['Continent']=X['Continent'].apply(lambda x: Continent_dic[x])
# replacing Entry Criteria by numeric values 
Entry_Criteria_dic={}
count=0

for criteria in X['Entry Criteria'].unique():
    if criteria=='Unknown':
        Entry_Criteria_dic[criteria]=-1
    else:
        Entry_Criteria_dic[criteria]=count
        count+=1
X['Entry Criteria']=X['Entry Criteria'].apply(lambda x: Entry_Criteria_dic[x])

C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [263]:
X.head()

,ISO,Region,Continent,Start Year,Entry Criteria,Start Month,Start Day,avg_rain,sum_rain,max_rain,stddev_rain,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_avg,last_10_days_max,last_20_days_avg,last_20_days_max
0,0,0,0,1906,0,5.0,14.0,13.00,13.0,13.0,0.00,478.000000,39.833333,117.0,22.181818,117.0,26.047619,117.0
1,0,0,0,1906,0,4.0,20.0,0.00,0.0,0.0,0.00,274.000000,22.833333,107.0,12.454545,107.0,6.761905,107.0
2,0,0,0,1926,0,3.0,20.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,1.909091,10.0,9.619048,55.0
3,1,1,1,1926,1,7.0,19.0,132.19,277585.0,3353.0,259.28,1367.419998,113.951667,4854.0,106.884545,4854.0,107.260476,4854.0
4,2,2,2,1927,2,11.0,1.0,48.87,733.0,230.0,70.69,380.240002,31.686667,250.0,33.875455,264.0,38.142381,390.0


In [264]:
# normalizing rain data 

list_rain_features=[' avg_rain', 'sum_rain','max_rain', 'stddev_rain', 'last_5_days_sum',
       'last_5_days_avg', 'last_5_days_max', 'last_10_days_avg', 
        'last_10_days_max', 'last_20_days_avg', 'last_20_days_max']

mean_normalization={}
std_normalization={}
for feature in list_rain_features:
    mean_feature=X[feature].mean()
    mean_normalization[feature]=mean_feature
    std_feature=X[feature].std()
    std_normalization[feature]=std_feature
    X[feature]=(X[feature]-mean_feature)/std_feature
    

## normalizing y data 
mean_y=y.mean()
std_y=y.std()
y=(y-mean_y)/std_y

C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [265]:
X.head()

,ISO,Region,Continent,Start Year,Entry Criteria,Start Month,Start Day,avg_rain,sum_rain,max_rain,stddev_rain,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_avg,last_10_days_max,last_20_days_avg,last_20_days_max
0,0,0,0,1906,0,5.0,14.0,-0.492165,-0.265732,-0.752995,-0.803105,-0.457521,-0.429116,-0.799729,-0.720740,-0.774820,-0.696134,-0.708456
1,0,0,0,1906,0,4.0,20.0,-0.570164,-0.265823,-0.768025,-0.803105,-0.715960,-0.636933,-0.807619,-0.868881,-0.781202,-1.039126,-0.713156
2,0,0,0,1926,0,3.0,20.0,-0.570164,-0.265823,-0.768025,-0.803105,-1.063080,-0.916059,-0.892032,-1.029481,-0.843112,-0.988312,-0.737597
3,1,1,1,1926,1,7.0,19.0,0.222963,1.680542,3.108564,0.844999,0.669251,0.476943,2.937338,0.569231,2.248550,0.748219,1.518009
4,2,2,2,1927,2,11.0,1.0,-0.276949,-0.260684,-0.502110,-0.353766,-0.581369,-0.528705,-0.694804,-0.542653,-0.680998,-0.481031,-0.580141


In [266]:
#X_train=X[X['Start Year']<=2015]
#y_train=y[X['Start Year']<=2015]
#print('X_train ' , X_train.shape)
#print('y_train ' , X_train.shape)

#X_test=X[X['Start Year']>2015]
#y_test=y[X['Start Year']>2015]

#print('X_test', X_test.shape)
#print('y_test', X_test.shape)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [267]:
print('X_train ' , X_train.shape)
print('y_train ' , X_train.shape)
print('X_test', X_test.shape)
print('y_test', X_test.shape)

X_train  (2692, 18)
y_train  (2692, 18)
X_test (1327, 18)
y_test (1327, 18)


In [268]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,max_error
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import max_error, explained_variance_score

In [269]:
regr = RandomForestRegressor(n_estimators=100, min_samples_split=15,random_state=2)
regr.fit(X_train, y_train)
y_pred=regr.predict(X_test)

In [270]:
# he coefficient of determination R^2 of the prediction
print('The best possible score is 1.0')
regr.score(X_test,y_test)

The best possible score is 1.0


-0.0411615218721435

In [271]:
mean_absolute_error(y_test, y_pred)*std_y+mean_y

3269518.578678439

In [272]:
max_error(y_test, y_pred)*std_y+mean_y

132522349.53622061

In [273]:
print('The best possible score is 1.0')

explained_variance_score(y_test, y_pred)

The best possible score is 1.0


-0.034445311692482106

In [274]:
# prediction aggregation by region and by year
Y_agg=pd.DataFrame()
Y_agg['Total Affected']=y_test*std_y+mean_y
Y_agg['predicted total Affected']=y_pred*std_y+mean_y
Y_agg['prediction error']=(y_test-y_pred)*std_y+mean_y
Y_agg['ISO']=X['ISO']
Y_agg['Region']=X['Region']
Y_agg['Continent']=X['Continent']
Y_agg['Start Year']=X['Start Year']
Y_agg['Entry Criteria']=X['Entry Criteria']
Y_agg['Start Month']=X['Start Month']
Y_agg['Start Day']=X['Start Day']
Y_agg

,Total Affected,predicted total Affected,prediction error,ISO,Region,Continent,Start Year,Entry Criteria,Start Month,Start Day
2295,2.261250e+03,8.653626e+04,1.148494e+06,103,13,0,2005,2,8.0,4.0
1204,3.500000e+04,4.679457e+05,7.998229e+05,33,2,2,1995,2,8.0,17.0
3644,1.647000e+03,1.179619e+05,1.116454e+06,11,3,3,2019,2,4.0,7.0
1953,1.000000e+04,1.803250e+05,1.062444e+06,136,15,2,2005,6,1.0,27.0
1226,1.010000e+02,2.789159e+06,-1.556290e+06,2,2,2,1996,3,4.0,23.0
...,...,...,...,...,...,...,...,...,...,...
1723,3.100000e+02,5.358429e+04,1.179494e+06,60,3,3,2001,3,4.0,25.0
2149,1.750000e+03,5.248933e+06,-4.014414e+06,38,1,1,2005,2,8.0,9.0
168,6.235502e+06,2.484242e+06,4.984029e+06,39,4,1,1959,2,7.0,20.0
3723,2.502000e+03,3.552756e+04,1.199743e+06,127,14,2,2017,6,1.0,20.0


In [275]:
Y_final= Y_agg[['Region','Start Year','predicted total Affected','Total Affected']].groupby(by=['Region','Start Year']).sum()
Y_final.head()

predicted total Affected  Total Affected
Region Start Year                                          
0      1906                    3.514422e+06    15299.367647
       1930                    3.775507e+06    15299.367647
       1956                    4.296326e+06    15299.367647
       1958                    5.220454e+06    15299.367647
       1971                    1.832142e+06      600.000000

In [276]:
Y_final['absolute difference']=(Y_final['predicted total Affected']-Y_final['Total Affected']).apply(lambda x: abs(x))

In [277]:
Y_final

predicted total Affected  Total Affected  \
Region Start Year                                             
0      1906                    3.514422e+06    15299.367647   
       1930                    3.775507e+06    15299.367647   
       1956                    4.296326e+06    15299.367647   
       1958                    5.220454e+06    15299.367647   
       1971                    1.832142e+06      600.000000   
...                                     ...             ...   
18     2020                    2.331746e+04      100.000000   
19     1990                    2.406839e+05      280.000000   
       2019                    1.527614e+05     6750.000000   
21     2008                    1.306819e+05      942.500000   
       2014                    9.314107e+04      360.000000   

                   absolute difference  
Region Start Year                       
0      1906               3.499122e+06  
       1930               3.760207e+06  
       1956               4.281026e+06  
       1958               5.205155e+06  
       1971               1.831542e+06  
...                                ...  
18     2020               2.321746e+04  
19     1990               2.404039e+05  
       2019               1.460114e+05  
21     2008               1.297394e+05  
       2014               9.278107e+04  

[581 rows x 3 columns]

In [278]:
Y_final['absolute difference'].mean()

3933633.1038038447

# Predicting flood events :

In [454]:
flood_table_full_date=flood_table[flood_table['Start Day'].notnull()]
print(flood_table_full_date.shape)

(4365, 41)


In [455]:
total_data=flood_table_full_date.merge(precipitation,how='right',left_on=['ISO','Start Year','Start Month','Start Day'],right_on=['ISO','Start Year','Start Month','Start Day'])
print(total_data.shape)
print(total_data.columns)

(5112885, 56)
Index(['Dis No', 'Year', 'Seq', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Event Name', 'Entry Criteria',
       'Country', 'ISO', 'Region', 'Continent', 'Location', 'Origin',
       'Associated Dis', 'Associated Dis2', 'OFDA Response', 'Appeal',
       'Declaration', 'Aid Contribution', 'Dis Mag Value', 'Dis Mag Scale',
       'Latitude', 'Longitude', 'River Basin', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths',
       'No Injured', 'No Affected', 'No Homeless', 'Total Affected',
       'Reconstruction Costs ('000 US$)', 'Insured Damages ('000 US$)',
       'Total Damages ('000 US$)', 'CPI', 'date', ' avg_rain', 'sum_rain',
       'max_rain', 'stddev_rain', 'station_count', 'last_5_days_sum',
       'last_5_days_avg', 'last_5_days_max', 'last_10_days_sum',
       'last_10_days_avg', 'last_10_days_max', 'last_20_days_sum',
       'last_20_days_avg', 'last_20_days_max'],
      dt

In [466]:
total_data['flood']=total_data['Dis No'].isnull()

In [467]:
# adding a features that indicated if there is a flood 
total_data['flood']=total_data['flood'].apply(lambda x: 0 if x else 1 )
total_data

,Dis No,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Event Name,Entry Criteria,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,OFDA Response,Appeal,Declaration,Aid Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI,date,avg_rain,sum_rain,max_rain,stddev_rain,station_count,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_sum,last_10_days_avg,last_10_days_max,last_20_days_sum,last_20_days_avg,last_20_days_max,flood
0,1906-0023-BEL,1906.0,23.0,Natural,Hydrological,Flood,NaN,NaN,Govern,Belgium,BEL,Western Europe,Europe,Louvain region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1906,5.0,14.0,1906.0,5.0,14.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.522300,1906-05-14,13.00,13.0,13.0,0.00,1,478.000000,39.833333,117.0,488.000000,22.181818,117.0,1094.000000,26.047619,117.0,1
1,1926-0008-IND,1926.0,8.0,Natural,Hydrological,Flood,Riverine flood,NaN,OFDA,India,IND,Southern Asia,Asia,Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1926,7.0,19.0,1926.0,7.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.940236,1926-07-19,132.19,277585.0,3353.0,259.28,2100,1367.419998,113.951667,4854.0,2351.459991,106.884545,4854.0,4504.939987,107.260476,4854.0,1
2,1927-0012-DZA,1927.0,12.0,Natural,Hydrological,Flood,NaN,NaN,Kill,Algeria,DZA,Northern Africa,Africa,Mostaganem,NaN,NaN,NaN,NaN,No,No,NaN,NaN,Km2,NaN,NaN,NaN,1927,11.0,1.0,1927.0,11.0,1.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.809780,1927-11-01,48.87,733.0,230.0,70.69,15,380.240002,31.686667,250.0,745.260006,33.875455,264.0,1601.980009,38.142381,390.0,1
3,1928-0018-IND,1928.0,18.0,Natural,Hydrological,Flood,NaN,NaN,Kill,India,IND,Southern Asia,Asia,"Godavari, Vizagapatnam",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1928,10.0,19.0,1928.0,10.0,19.0,36.0,NaN,NaN,2000.0,2000.0,NaN,NaN,NaN,6.731507,1928-10-19,26.76,58363.0,1090.0,77.33,2181,423.679996,35.306666,1496.0,965.719997,43.896364,2743.0,1861.719986,46.543000,3162.0,1
4,1930-0021-BEL,1930.0,21.0,Natural,Hydrological,Flood,Riverine flood,NaN,Govern,Belgium,BEL,Western Europe,Europe,"Termonde, Anvers, Charleroi, Bruxelles",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Km2,NaN,NaN,NaN,1930,11.0,22.0,1930.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.548869,1930-11-22,118.00,118.0,118.0,0.00,1,1074.000000,89.500000,200.0,1330.000000,60.454545,200.0,2318.000000,55.190476,213.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,12.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-18,0.00,0.0,0.0,0.00,1,128.000000,10.666667,56.0,448.000000,20.363636,160.0,672.000000,16.000000,160.0,0
5112881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,12.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-19,0.00,0.0,0.0,0.00,1,128.000000,10.666667,56.0,448.000000,20.363636,160.0,672.000000,16.000000,160.0,0
5112882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,12.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-20,0.00,0.0,0.0,0.00,1,122.000000,10.166667,56.0,448.000000,20.363636,160.0,672.000000,16.000000,160.0,0
5112883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,12.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-21,0.00,0.0,0.0,0.00,1,112.000000,9.333333,56.0,448.000000,20.363636,160.0,672.000000,16.000000,160.0,0


In [468]:
total_data.columns

Index(['Dis No', 'Year', 'Seq', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Event Name', 'Entry Criteria',
       'Country', 'ISO', 'Region', 'Continent', 'Location', 'Origin',
       'Associated Dis', 'Associated Dis2', 'OFDA Response', 'Appeal',
       'Declaration', 'Aid Contribution', 'Dis Mag Value', 'Dis Mag Scale',
       'Latitude', 'Longitude', 'River Basin', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths',
       'No Injured', 'No Affected', 'No Homeless', 'Total Affected',
       'Reconstruction Costs ('000 US$)', 'Insured Damages ('000 US$)',
       'Total Damages ('000 US$)', 'CPI', 'date', ' avg_rain', 'sum_rain',
       'max_rain', 'stddev_rain', 'station_count', 'last_5_days_sum',
       'last_5_days_avg', 'last_5_days_max', 'last_10_days_sum',
       'last_10_days_avg', 'last_10_days_max', 'last_20_days_sum',
       'last_20_days_avg', 'last_20_days_max', 'flood'],
      dtype='

In [469]:
X=total_data[['ISO','Region', 'Continent','Start Year','Start Month','Start Day',' avg_rain', 'sum_rain',
       'max_rain', 'stddev_rain', 'last_5_days_sum',
       'last_5_days_avg', 'last_5_days_max', 'last_10_days_sum',
       'last_10_days_avg', 'last_10_days_max', 'last_20_days_sum',
       'last_20_days_avg', 'last_20_days_max']]
y=total_data[['flood']]

In [470]:
# replacing ISO by numeric values 
ISO_dic={}
count=0
for iso in X['ISO'].unique():
    ISO_dic[iso]=count
    count+=1
X['ISO']=X['ISO'].apply(lambda x: ISO_dic[x])

Region_dic={}
count=0
for region in X['Region'].unique():
    Region_dic[region]=count
    count+=1
X['Region']=X['Region'].apply(lambda x: Region_dic[x])

Continent_dic={}
count=0
for continent in X['Continent'].unique():
    Continent_dic[continent]=count
    count+=1
X['Continent']=X['Continent'].apply(lambda x: Continent_dic[x])

C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GAMDOU\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [471]:
X.head(30)
# I have to add the same things for other samples (variables otherwisse RF will identify them )

,ISO,Region,Continent,Start Year,Start Month,Start Day,avg_rain,sum_rain,max_rain,stddev_rain,last_5_days_sum,last_5_days_avg,last_5_days_max,last_10_days_sum,last_10_days_avg,last_10_days_max,last_20_days_sum,last_20_days_avg,last_20_days_max
0,0,0,0,1906,5.0,14.0,13.00,13.0,13.0,0.00,478.000000,39.833333,117.0,488.000000,22.181818,117.0,1094.000000,26.047619,117.0
1,1,1,1,1926,7.0,19.0,132.19,277585.0,3353.0,259.28,1367.419998,113.951667,4854.0,2351.459991,106.884545,4854.0,4504.939987,107.260476,4854.0
2,2,2,2,1927,11.0,1.0,48.87,733.0,230.0,70.69,380.240002,31.686667,250.0,745.260006,33.875455,264.0,1601.980009,38.142381,390.0
3,1,1,1,1928,10.0,19.0,26.76,58363.0,1090.0,77.33,423.679996,35.306666,1496.0,965.719997,43.896364,2743.0,1861.719986,46.543000,3162.0
4,0,0,0,1930,11.0,22.0,118.00,118.0,118.0,0.00,1074.000000,89.500000,200.0,1330.000000,60.454545,200.0,2318.000000,55.190476,213.0
5,3,3,3,1950,5.0,5.0,21.89,21534.0,615.0,65.58,158.340000,13.195000,615.0,281.500002,12.795455,615.0,660.260001,15.720476,787.0
6,4,4,1,1950,8.0,14.0,201.00,201.0,201.0,0.00,2542.000000,254.200000,399.0,2982.000000,165.666667,399.0,4840.000000,134.444444,450.0
7,1,1,1,1950,7.0,11.0,130.34,306037.0,2372.0,216.94,1338.839996,111.570000,3866.0,2011.219994,91.419091,3866.0,3065.440002,72.986667,3866.0
8,4,4,1,1951,8.0,28.0,22.22,2088.0,294.0,56.69,490.159992,40.846666,1357.0,776.799999,35.309091,1357.0,1679.400002,39.985714,1417.0
9,5,5,0,1952,8.0,15.0,61.42,2088.0,539.0,116.91,323.139994,26.928333,539.0,907.799997,41.263636,539.0,1491.039996,35.500952,539.0


In [472]:
y

,flood
0,1
1,1
2,1
3,1
4,1
...,...
5112880,0
5112881,0
5112882,0
5112883,0


In [463]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
import seaborn as sns


In [464]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model=RandomForestClassifier(n_estimators=20, min_samples_split=4, random_state=0)
model.fit(X_train, y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
balanced_accuracy_score(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
cm=confusion_matrix(y_test, y_pred,labels=[0,1])

In [ ]:
plt.figure(figsize=(8,6))

ax =sns.heatmap(cm, annot=True,fmt='.2f',cmap="Blues")
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
y_prob=model.predict_proba(X_test)
y_prob

In [ ]:
df_prob=pd.DataFrame(y_prob, index=model.classes_)
df_prob